In [ ]:
import requests
from bs4 import BeautifulSoup
import json

In [14]:
test_url = "https://biblehub.com/1_peter/1-1.htm"
r = requests.get(test_url)


In [15]:
import asyncio, httpx

urls = [f"https://biblehub.com/1_peter/{i}-1.htm" for i in range(1, 6)]

async def main():
    limits = httpx.Limits(max_connections=100, max_keepalive_connections=20)
    timeout = httpx.Timeout(8.0)
    async with httpx.AsyncClient(http2=True, limits=limits, timeout=timeout, headers={
        "User-Agent": "Mozilla/5.0",
        "Accept-Encoding": "gzip, deflate, br",
    }) as client:
        sem = asyncio.Semaphore(20)  # control concurrency

        async def fetch(url):
            async with sem:
                r = await client.get(url)
                r.raise_for_status()
                return r.text

        return await asyncio.gather(*(fetch(u) for u in urls))

html_pages = asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [13]:
soup = BeautifulSoup(r.text, "html.parser")
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "//www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="//www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   1 Peter 1:1 Peter, an apostle of Jesus Christ, To the elect who are exiles of the Dispersion throughout Pontus, Galatia, Cappadocia, Asia, and Bithynia, chosen
  </title>
  <link href="https://biblehub.com/1_peter/1-1.htm" rel="canonical"/>
  <link href="/new10.css" media="Screen" rel="stylesheet" type="text/css"/>
  <meta content="https://biblehub.com/visuals/11/60_1Pe_01_01.jpg" property="og:image"/>
  <meta content="1 Peter 1:1 - Greetings from Peter" property="og:title"/>
  <meta content="Bible Hub" property="og:site_name"/>
  <meta content="Peter, an apostle of Jesus Christ, To the elect who are exiles of the Dispersion throughout Pontus, Galatia, Cappadocia, Asia, a

In [ ]:
base = "http://60day.adventurechurch.org/"
results = {}

for day in range(1, 61):
    url = f"{base}day-1/" if day == 1 else f"{base}day{day}/"
    r = requests.get(url)
    if r.status_code != 200:
        results[day] = None
        print(day, "ah ah ah")
        continue

    soup = BeautifulSoup(r.text, "html.parser")

    # find the "Read ..." line
    tag = soup.find("p")
    read_text = None
    if tag and "Read" in tag.get_text():
        read_text = tag.get_text(strip=True)

    results[day] = read_text

# save to json
with open("readings.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)


  0%|          | 0/60 [00:00<?, ?it/s]

100%|██████████| 60/60 [01:02<00:00,  1.04s/it]


In [6]:
print(results)

{1: 'Read Matthew 1-4', 2: None, 3: 'Read Matthew 9-12', 4: 'Read Matthew 13-16', 5: 'Read Matthew 17-20', 6: 'Read Matthew 21-24', 7: 'Read Matthew 25-28', 8: 'Read Mark 1-4', 9: 'Read Mark 5-8', 10: 'Read Mark 9-12', 11: 'Read Mark 13-16', 12: None, 13: 'Read Luke 4-7', 14: 'Read Luke 8-10', 15: 'Read Luke 11-13', 16: 'Read Luke 14-17', 17: None, 18: 'Read Luke 22-24', 19: 'Read John 1-4', 20: 'Read John 5-7', 21: 'Read John 8-10', 22: 'Read John 11-13', 23: 'Read John 14-17', 24: 'Read John 18-21', 25: 'Read Acts 1-4', 26: 'Read Acts 5 – 7', 27: 'Read Acts 8 – 11', 28: 'Read Acts 12-15', 29: 'Read Acts 16-18', 30: 'Read Acts 19-23', 31: 'Read Acts 24-28', 32: 'Read Romans 1-4', 33: 'Read Romans 5-8', 34: 'Read Romans 9-12', 35: 'Read Romans 13-16', 36: 'Read I Corinthians 1-5', 37: 'Read 1 Corinthians 6-10', 38: 'Read 1 Corinthians 11-16', 39: 'Read 2 Corinthians 1-7', 40: 'Read 2 Corinthians 8-13', 41: 'Read Galatians 1-3', 42: 'Read Galatians 4-6', 43: 'Read Ephesians 1-3', 44: 'R